1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
Для парсинга использовать xpath. Структура данных должна содержать:
название источника,
наименование новости,
ссылку на новость,
дата публикации

2)Сложить все новости в БД

In [2]:
from pprint import pprint
from lxml import html
import requests
import re
import pandas as pd
from datetime import datetime, date, time
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

yandex

In [ ]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
                        Chrome/76.0.3809.100 Safari/537.36 OPR/63.0.3368.53'
    }
main_link = 'https://yandex.ru/news/' 
req = requests.get(main_link,headers=headers)
root_yandex = html.fromstring(req.text)

news_yandex_text=[]
news_yandex_urls=[]
driver = webdriver.PhantomJS('C:\\Users\\krilatiy\\phantomjs-2.1.1-windows\\bin\\phantomjs')
driver.get(main_link)
time.sleep(3)
content = driver.find_elements_by_xpath('//h2[@class="story__title"]/a')
for elem in content:
        news_yandex_text.append(elem.text)
        news_yandex_urls.append(elem.get_attribute("href"))
driver.close()

source=[]
date=[]
for i,elem in enumerate(news_yandex_urls):
    source.append('https://yandex.ru/news/')
    date.append(str(datetime.now()))
    
s_text = pd.Series(news_yandex_text)
s_urls = pd.Series(news_yandex_urls)
s_source = pd.Series(source)
s_date = pd.Series(date)
df = pd.DataFrame()
df['url'] = s_urls
df['news_text'] = s_text
df['source'] = s_source
df['datetime'] = s_date

mail

In [ ]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
                        Chrome/76.0.3809.100 Safari/537.36 OPR/63.0.3368.53'
    }

main_link = 'https://www.mail.ru/' 
req = requests.get(main_link,headers=headers)
root_mail = html.fromstring(req.text)
news_block_mail = root_mail.xpath('//div[contains(@class,I4bwWHu)]')
text = news_block_mail[0].xpath('//a/text()')

first_news_text = root_mail.xpath('//div[contains(@class,news-item_main)]/h3/text()')

news_text_mail=[]
news_text_mail.append(first_news_text[0])
for elem in text:
    if len(elem) > 15:
        elem = re.sub('\xa0'," ",elem)
        news_text_mail.append(elem) 
news_text_mail

urls = news_block_mail[0].xpath('//a/@href')

news_urls_mail=[]
for url in urls:
    if url.find('https://news.mail.ru/') >= 0 or \
       url.find('https://sportmail.ru/') >= 0 or \
       url.find('https://auto.mail.ru/') >= 0 or \
       url.find('https://kino.mail.ru/') >= 0 :
        news_urls_mail.append(url)
news_urls_mail

len(news_text_mail)==len(news_urls_mail)

source=[]
date=[]
for i,elem in enumerate(news_urls_mail):
    source.append('mail.ru')
    date.append(str(datetime.now()))
    
    
s_text = pd.Series(news_text_mail)
s_urls = pd.Series(news_urls_mail)
s_source = pd.Series(source)
s_date = pd.Series(date)
df = pd.DataFrame()
df['url'] = s_urls
df['news_text'] = s_text
df['source'] = s_source
df['datetime'] = s_date
df.head()


lenta

In [ ]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
                        Chrome/76.0.3809.100 Safari/537.36 OPR/63.0.3368.53'
    }
main_link = 'https://www.lenta.ru/' 
req = requests.get(main_link,headers=headers)
root_lenta = html.fromstring(req.text)
first_news_lenta = root_lenta.xpath('//div[@class="first-item"]')

first_text = first_news_lenta[0].xpath('//h2//a/text()')
first_news_text = re.sub('\xa0'," ",first_text[0])

news_block_lenta = root_lenta.xpath('//section[contains(@class,js-top-seven)]')
text = news_block_lenta[0].xpath('//div[contains(@class,item)]/a/text()')

news_text_lenta=[]
news_text_lenta.append(first_news_text)
for elem in text[1:11]:
    if len(elem) > 15:
        elem = re.sub('\xa0'," ",elem)
        news_text_lenta.append(elem) 
news_text_lenta

urls = news_block_lenta[0].xpath('//div[contains(@class,item)]/a/@href')

news_urls_lenta=[]
for url in urls:
    if url.find('/news/20') >= 0:
        url = 'https://lenta.ru' + url
        news_urls_lenta.append(url)
news_urls_lenta = news_urls_lenta[:10]
news_urls_lenta

source=[]
date=[]
for i,elem in enumerate(news_urls_lenta):
    source.append('lenta.ru')
    date.append(str(datetime.now()))
    
s_text = pd.Series(news_text_lenta)
s_urls = pd.Series(news_urls_lenta)
s_source = pd.Series(source)
s_date = pd.Series(date)
df = pd.DataFrame()
df['url'] = s_urls
df['news_text'] = s_text
df['source'] = s_source
df['datetime'] = s_date